In [4]:
from fastapi import FastAPI,HTTPException,Request
from google.api_core.exceptions import GoogleAPICallError
from google.cloud import bigquery
from dotenv import load_dotenv
import pandas as pd
from typing import Optional,List,Literal
import numpy as np
from google.cloud.bigquery import ScalarQueryParameter, QueryJobConfig
import os
import requests
os.chdir("..")
load_dotenv()

True

In [6]:
os.getenv("GOOGLE_APPLICATION_CREDENTIALS"),os.getcwd()

('sibr-market-815c8fa2699d.json',
 '/Users/sigvardbratlie/PycharmProjects/sibr-market-frontend')

In [7]:
client = bigquery.Client()

In [8]:
def run_query(query: str, params: Optional[List[ScalarQueryParameter]] = None) -> pd.DataFrame:
    """
    Kjører en BigQuery-spørring sikkert med parametere og returnerer resultatet som en Pandas DataFrame.
    """
    if not client:
        raise HTTPException(status_code=500, detail="BigQuery client is not available.")
    try:
        # Konfigurerer spørringen med parametere for å unngå SQL-injeksjon
        job_config = QueryJobConfig(query_parameters=params) if params else None

        # Kjører spørringen
        print(f"Executing query: {query}")
        query_job = client.query(query, job_config=job_config)

        # Venter på at jobben skal fullføre og henter resultatene
        results = query_job.to_dataframe()
        return results
    except GoogleAPICallError as e:
        # Håndterer spesifikke API-feil fra Google Cloud og sender detaljene tilbake
        print(f"A BigQuery API error occurred: {e}")
        raise HTTPException(status_code=500, detail=f"BigQuery API error: {str(e)}")
    except Exception as e:
        # Håndterer andre generelle feil og sender detaljene tilbake
        print(f"An unexpected error occurred: {e}")
        raise HTTPException(status_code=500, detail=f"An internal error occurred: {str(e)}")
def get_dashboard_stats(
        request: Request,
        period: Literal["month", "quarter"] = "month",
        page: int = 1,
        limit: int = 50
):
    """
    Fetches aggregated KPIs and a paginated list of homes based on dynamic filters.
    Calculates change from the previous period (month or quarter).
    """
    filters = request.query_params

    # Bygg den dynamiske WHERE-betingelsen og parametere
    conditions = []
    query_params = []
    for key, value in filters.items():
        # Ignorerer parametere som ikke er ment for filtrering
        if key not in ["period", "page", "limit"] and value:
            conditions.append(f"LOWER(CAST(`{key}` AS STRING)) LIKE @{key}")
            query_params.append(ScalarQueryParameter(key, "STRING", f"%{value.lower()}%"))

    where_clause = f"WHERE {' AND '.join(conditions)}" if conditions else ""

    # Definer tidsenheten for BigQuery
    bq_period = "MONTH" if period == "month" else "QUARTER"

    # SQL for å hente KPI-er for nåværende og forrige periode i ett kall
    kpi_query = f"""
    WITH FilteredData AS (
        SELECT
            price, usable_area, build_year, last_updated,
            -- Definerer perioden for hver rad
            DATE_TRUNC(DATE(last_updated), {bq_period}) as period_start
        FROM `sibr-market.agent.homes`
        {where_clause}
    ),
    LatestPeriod AS (
        SELECT MAX(period_start) as current_period_start FROM FilteredData
    )
    SELECT
        period_type,
        ROUND(AVG(price)) as avg_price,
        ROUND(AVG(usable_area)) as avg_sqm,
        ROUND(AVG(build_year)) as avg_build_year,
        COUNT(*) as n_samples
    FROM (
        SELECT
            'current' as period_type, F.*
        FROM FilteredData F, LatestPeriod L
        WHERE F.period_start = L.current_period_start
        UNION ALL
        SELECT
            'previous' as period_type, F.*
        FROM FilteredData F, LatestPeriod L
        WHERE F.period_start = DATE_SUB(L.current_period_start, INTERVAL 1 {bq_period})
    )
    GROUP BY period_type
    """

    kpi_df = run_query(kpi_query, params=query_params)
    kpis = {row['period_type']: row for row in kpi_df.to_dict('records')}

    # SQL for å hente tabelldata med paginering
    columns = [
        "item_id", "address", "municipality", "county", "price", "price_pr_sqm",
        "property_type", "usable_area", "bedrooms", "build_year",
        "last_updated", "dealer", "url"
    ]
    valid_columns_str = ", ".join([f"`{col}`" for col in columns])
    offset = (page - 1) * limit

    homes_query = f"""
    SELECT {valid_columns_str} FROM `sibr-market.agent.homes`
    {where_clause}
    ORDER BY last_updated DESC
    LIMIT @limit OFFSET @offset
    """

    # Legg til pagineringsparametere (de må være bakerst)
    pagination_params = [
        ScalarQueryParameter("limit", "INT64", limit),
        ScalarQueryParameter("offset", "INT64", offset)
    ]

    homes_df = run_query(homes_query, params=query_params + pagination_params)
    homes = homes_df.replace({np.nan: None, pd.NaT: None}).to_dict('records')

    return {"kpis": kpis, "homes": homes}

In [9]:
get_dashboard_stats()

TypeError: get_dashboard_stats() missing 1 required positional argument: 'request'

In [1]:
pakker = """
annotated-types==0.7.0
anyio==4.9.0
appnope==0.1.4
argon2-cffi==25.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==3.0.0
async-lru==2.0.5
attrs==25.3.0
babel==2.17.0
beautifulsoup4==4.13.4
bleach==6.2.0
cachetools==5.5.2
certifi==2025.7.14
cffi==1.17.1
charset-normalizer==3.4.2
click==8.1.8
comm==0.2.3
db-dtypes==1.4.3
debugpy==1.8.15
decorator==5.2.1
defusedxml==0.7.1
dnspython==2.7.0
email_validator==2.2.0
exceptiongroup==1.3.0
executing==2.2.0
fastapi==0.116.1
fastapi-cli==0.0.8
fastapi-cloud-cli==0.1.5
fastjsonschema==2.21.1
fqdn==1.5.1
google-api-core==2.25.1
google-auth==2.40.3
google-cloud==0.34.0
google-cloud-bigquery==3.35.1
google-cloud-bigquery-storage==2.32.0
google-cloud-core==2.4.3
google-crc32c==1.7.1
google-resumable-media==2.7.2
googleapis-common-protos==1.70.0
grpcio==1.74.0
grpcio-status==1.74.0
h11==0.16.0
httpcore==1.0.9
httptools==0.6.4
httpx==0.28.1
idna==3.10
importlib_metadata==8.7.0
iniconfig==2.1.0
ipykernel==6.30.0
ipython==8.18.1
isoduration==20.11.0
jedi==0.19.2
Jinja2==3.1.6
json5==0.12.0
jsonpointer==3.0.0
jsonschema==4.25.0
jsonschema-specifications==2025.4.1
jupyter-events==0.12.0
jupyter-lsp==2.2.6
jupyter_client==8.6.3
jupyter_core==5.8.1
jupyter_server==2.16.0
jupyter_server_terminals==0.5.3
jupyterlab==4.4.5
jupyterlab_pygments==0.3.0
jupyterlab_server==2.27.3
lark==1.2.2
markdown-it-py==3.0.0
MarkupSafe==3.0.2
matplotlib-inline==0.1.7
mdurl==0.1.2
mistune==3.1.3
nbclient==0.10.2
nbconvert==7.16.6
nbformat==5.10.4
nest-asyncio==1.6.0
notebook==7.4.4
notebook_shim==0.2.4
numpy==2.0.2
overrides==7.7.0
packaging==25.0
pandas==2.3.1
pandocfilters==1.5.1
parso==0.8.4
pexpect==4.9.0
platformdirs==4.3.8
pluggy==1.6.0
prometheus_client==0.22.1
prompt_toolkit==3.0.51
proto-plus==1.26.1
protobuf==6.31.1
psutil==7.0.0
ptyprocess==0.7.0
pure_eval==0.2.3
pyarrow==21.0.0
pyasn1==0.6.1
pyasn1_modules==0.4.2
pycparser==2.22
pydantic==2.11.7
pydantic_core==2.33.2
Pygments==2.19.2
pytest==8.4.1
python-dateutil==2.9.0.post0
python-dotenv==1.1.1
python-json-logger==3.3.0
python-multipart==0.0.20
pytz==2025.2
PyYAML==6.0.2
pyzmq==27.0.0
referencing==0.36.2
requests==2.32.4
rfc3339-validator==0.1.4
rfc3986-validator==0.1.1
rfc3987-syntax==1.1.0
rich==14.1.0
rich-toolkit==0.14.9
rignore==0.6.4
rpds-py==0.26.0
rsa==4.9.1
Send2Trash==1.8.3
sentry-sdk==2.33.2
shellingham==1.5.4
six==1.17.0
sniffio==1.3.1
soupsieve==2.7
stack-data==0.6.3
starlette==0.47.2
terminado==0.18.1
tinycss2==1.4.0
tomli==2.2.1
tornado==6.5.1
traitlets==5.14.3
typer==0.16.0
types-python-dateutil==2.9.0.20250708
typing-inspection==0.4.1
typing_extensions==4.14.1
tzdata==2025.2
uri-template==1.3.0
urllib3==2.5.0
uvicorn==0.35.0
uvloop==0.21.0
watchfiles==1.1.0
wcwidth==0.2.13
webcolors==24.11.1
webencodings==0.5.1
websocket-client==1.8.0
websockets==15.0.1
zipp==3.23.0

"""

In [2]:
pakker = pakker.split("\n")
pakker = [pakk.split("==")[0] for pakk in pakker]
#pakker = str(pakker).replace("[", "").replace("]", "").replace(",","").replace("'","")
packs = ""
for pakk in pakker:
    pakk = pakk.replace("'", "")
    packs += pakk + '\n'
print(packs)


annotated-types
anyio
appnope
argon2-cffi
argon2-cffi-bindings
arrow
asttokens
async-lru
attrs
babel
beautifulsoup4
bleach
cachetools
certifi
cffi
charset-normalizer
click
comm
db-dtypes
debugpy
decorator
defusedxml
dnspython
email_validator
exceptiongroup
executing
fastapi
fastapi-cli
fastapi-cloud-cli
fastjsonschema
fqdn
google-api-core
google-auth
google-cloud
google-cloud-bigquery
google-cloud-bigquery-storage
google-cloud-core
google-crc32c
google-resumable-media
googleapis-common-protos
grpcio
grpcio-status
h11
httpcore
httptools
httpx
idna
importlib_metadata
iniconfig
ipykernel
ipython
isoduration
jedi
Jinja2
json5
jsonpointer
jsonschema
jsonschema-specifications
jupyter-events
jupyter-lsp
jupyter_client
jupyter_core
jupyter_server
jupyter_server_terminals
jupyterlab
jupyterlab_pygments
jupyterlab_server
lark
markdown-it-py
MarkupSafe
matplotlib-inline
mdurl
mistune
nbclient
nbconvert
nbformat
nest-asyncio
notebook
notebook_shim
numpy
overrides
packaging
pandas
pandocfilters
pa